In [28]:
import cv2
import numpy as np
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

Using cache found in /home/v.lomonosov/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-6-3 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [29]:
!wget https://gkyars.ru/wp-content/uploads/2019/06/tov-080754.jpg -O molotok.jpg

--2024-06-03 11:59:21--  https://gkyars.ru/wp-content/uploads/2019/06/tov-080754.jpg
Resolving gkyars.ru (gkyars.ru)... 45.130.41.97
Connecting to gkyars.ru (gkyars.ru)|45.130.41.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105633 (103K) [image/jpeg]
Saving to: ‘molotok.jpg’

molotok.jpg         100%[===================>] 103.16K  --.-KB/s    in 0.1s    

2024-06-03 11:59:22 (862 KB/s) - ‘molotok.jpg’ saved [105633/105633]



In [30]:
img_path = 'molotok.jpg'
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img_rgb, (640, 640))
img_resized = img_resized / 255.0
img_resized = img_resized.transpose(2, 0, 1)
img_resized = np.expand_dims(img_resized, axis=0)
img_tensor = torch.from_numpy(img_resized).float()

In [31]:
with torch.no_grad():
    outputs = model(img_tensor)

outputs_np = [output.cpu().numpy() for output in outputs]


In [32]:
def postprocess_rknn(outputs, img_shape, conf_thres=0.25, iou_thres=0.45):
    predictions = outputs[0]
    boxes = predictions[:, :4]
    scores = predictions[:, 4]
    class_ids = predictions[:, 5].astype(int)

    boxes[:, 0] = boxes[:, 0] - boxes[:, 2] / 2
    boxes[:, 1] = boxes[:, 1] - boxes[:, 3] / 2
    boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
    boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

    img_h, img_w = img_shape[:2]
    boxes[:, [0, 2]] *= img_w / 640
    boxes[:, [1, 3]] *= img_h / 640

    mask = scores >= conf_thres
    boxes = boxes[mask]
    scores = scores[mask]
    class_ids = class_ids[mask]

    indices = cv2.dnn.NMSBoxes(
        boxes.tolist(), scores.tolist(), conf_thres, iou_thres
    )
    indices = indices.flatten() if len(indices) > 0 else []

    nms_boxes = boxes[indices]
    nms_scores = scores[indices]
    nms_class_ids = class_ids[indices]

    return nms_boxes, nms_scores, nms_class_ids

boxes, scores, class_ids = postprocess_rknn(outputs_np, img.shape)



In [33]:
for box, score, class_id in zip(boxes, scores, class_ids):
    x1, y1, x2, y2 = box
    label = f'{class_id}: {score:.2f}'
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imwrite('output_with_boxes.jpg', img)

np.save('output.npy', outputs_np)